In [3]:
!wget https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt

--2019-07-10 07:00:34--  https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4573338 (4.4M) [text/plain]
Saving to: ‘shakespeare_input.txt.1’

shakespeare_input.t 100%[===================>]   4.36M  5.09MB/s    in 0.9s    

2019-07-10 07:00:35 (5.09 MB/s) - ‘shakespeare_input.txt.1’ saved [4573338/4573338]



In [4]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RNN
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
filename = "./shakespeare_input.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
seq_length = 200
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 20):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# Model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# Checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, y, epochs=50, batch_size=128, callbacks=callbacks_list)

Using TensorFlow backend.


Total Characters:  4573338
Total Vocab:  41
Total Patterns:  228657


W0710 07:00:45.996554 139905521088384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0710 07:00:46.025842 139905521088384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0710 07:00:46.031201 139905521088384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0710 07:00:46.325721 139905521088384 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0710 07:00:46.334573 

Epoch 1/50
228657/228657 [==============================] - 456s 2ms/step - loss: 2.9253

Epoch 00001: loss improved from inf to 2.92532, saving model to weights-improvement-01-2.9253.hdf5
Epoch 2/50
228657/228657 [==============================] - 451s 2ms/step - loss: 2.7599

Epoch 00002: loss improved from 2.92532 to 2.75992, saving model to weights-improvement-02-2.7599.hdf5
Epoch 3/50
228657/228657 [==============================] - 451s 2ms/step - loss: 2.7028

Epoch 00003: loss improved from 2.75992 to 2.70285, saving model to weights-improvement-03-2.7028.hdf5
Epoch 4/50
228657/228657 [==============================] - 450s 2ms/step - loss: 2.6526

Epoch 00004: loss improved from 2.70285 to 2.65262, saving model to weights-improvement-04-2.6526.hdf5
Epoch 5/50
228657/228657 [==============================] - 453s 2ms/step - loss: 2.6078

Epoch 00005: loss improved from 2.65262 to 2.60780, saving model to weights-improvement-05-2.6078.hdf5
Epoch 6/50
228657/228657 [=============

In [0]:
#!gsutil -m cp -r  gs://proobj_1/ ./w* (for getting previously trained weights)

In [0]:
filename = "weights-improvement-05-2.7328.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [0]:
# For generating character level text
import os
import sys
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

In [0]:
!gsutil -m cp -r w* gs://proobj_1/